<font size=5  color=#003366> [LINMA2472] - Algorithms in Data Science <br>
Project 1: Networks </font> <br><br>
<font size=3  color=#003366>  Students: Iacopo Canetta, Giovanni Faldani
<br><br>

- <font size=4  color=#003366>Task 1: building the co-occurence network of characters </font> <br><br>
In this project we'll build a graph network representing the bonds between characters present in the classical novel "Treasure Island" by Robert Louis Stevenson